In [20]:
%pip install owlrl
%pip install rdflib
%pip install pyld

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pyld from https://files.pythonhosted.org/packages/44/cd/80760be197a4bd08e7c136ef4bcb4a2c63fc799d8d91f4c177b21183135e/PyLD-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict from https://files.pythonhosted.org/packages/04/13/d9839089b900fa7b479cce495d62110cddc4bd5630a04d8469916c0e79c5/frozendict-2.4.6-py311-none-any.whl.metadata
   ---------------------------------------- 0.0/70.9 kB ? eta -:--:--
   ---------------------------------------- 70.9/70.9 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [116]:
from rdflib.graph import Graph
from rdflib.namespace import RDF, Namespace
from rdflib.serializer import Serializer
from rdflib.term import BNode, Literal, URIRef
from rdflib.plugins.sparql import prepareQuery

In [21]:
import pathlib
from pyld import jsonld
import json

Serializing JSONLD Graph

```python
flattenedCrateMetadata = jsonld.flatten(crateMetadataRaw)

# write out the 
with pathlib.Path("metadata.json").open('w') as outfile:
	json.dump(flattenedCrateMetadata, outfile, indent=2)


# serialization 
normalized = jsonld.normalize(
    crateMetadataRaw, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})


with pathlib.Path("normalized.nquads").open('w') as outfile:
	outfile.write(normalized)
```

In [ ]:

with pathlib.Path("ro-crate-metadata.json").open('r') as jsonfile:
	crateMetadataRaw = json.load(jsonfile)

crateGUID = crateMetadataRaw['@id']
for elem in crateMetadataRaw['@graph']:
	elem['isPartOf'] = crateGUID

rocrateNode = crateMetadataRaw.copy()
del rocrateNode['@graph']

In [85]:
rocrateNode['@context']

{'EVI': 'https://w3id.org/EVI#', '@vocab': 'https://schema.org/'}

In [ ]:
context = {
	"EVI": 
}

In [164]:
g = Graph()

# load in all nodes of the graph
for elem in crateMetadataRaw['@graph']:
	elem['@context'] = rocrateNode['@context']
	expandedNode = jsonld.expand(elem)
	g.parse(data=json.dumps(expandedNode), format='json-ld')

# load in the metadata
g.parse(data=json.dumps(rocrateNode), format='json-ld')

<Graph identifier=Nc63ab907a9b64c61a6c579907e2b6218 (<class 'rdflib.graph.Graph'>)>

In [165]:
len(g.all_nodes())

631

In [166]:
# add the EVI ontology to the graph
g.parse(pathlib.Path('evi.owl'), format='xml')
EVI = Namespace("https://w3id.org/EVI#")
SCHEMA = Namespace("https://schema.org/")

In [97]:
# SPARQL query for all nodes

# rdflib query for datasets
datasetRDFLibNodes = [ s for s, p, o in g.triples((None,  RDF.type, EVI['Dataset']))] 
softwareRDFLibNodes = [ s for s, p, o in g.triples((None,  RDF.type, EVI['Software']))] 
computationRDFLibNodes = [ s for s, p, o in g.triples((None,  RDF.type, EVI['Computation']))] 
rocrateRDFLibNode = [ s for s, p, o in g.triples((None,  RDF.type, SCHEMA['Dataset']))] 


## Sparql query notes


  dataset? RDF.type EVI.Dataset;

In [127]:
RDF

Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')

In [ ]:
# sparql query
EVI = Namespace("https://w3id.org/EVI#")
sparqlQuery = prepareQuery("""
SELECT ?dataset ?pred ?obj
WHERE {
	 ?dataset ?pred ?obj .
	 
	 { ?dataset RDF:type EVI:Software .} UNION { ?dataset RDF:type EVI:Computation .}
}
""",

initNs={"RDF": RDF, "EVI": EVI, "SCHEMA": SCHEMA}

)

In [168]:
# execute query
sparqlResult = g.query(sparqlQuery)

# serialize results as json
# sparqlResult.serialize("query_output.json", format='json')

In [158]:
# create a new graph with the results
queryGraph = Graph()

for row in sparqlResult:
	queryGraph.add(row)

In [159]:
queryGraph.serialize('query_output.json', format='json-ld')

<Graph identifier=N380db6ef84a44fd5ae3a00f219bbdae2 (<class 'rdflib.graph.Graph'>)>

In [ ]:
help(g.obj)

Help on method subject_objects in module rdflib.graph:

subject_objects(predicate: 'Union[None, Path, _PredicateType]' = None, unique: 'bool' = False) -> 'Generator[Tuple[_SubjectType, _ObjectType], None, None]' method of rdflib.graph.Graph instance
    A generator of (optionally unique) (subject, object) tuples
    for the given predicate



In [163]:
help(g.transitive_objects)

Help on method transitive_objects in module rdflib.graph:

transitive_objects(subject: 'Optional[_SubjectType]', predicate: 'Optional[_PredicateType]', remember: 'Optional[Dict[Optional[_SubjectType], int]]' = None) -> 'Generator[Optional[_SubjectType], None, None]' method of rdflib.graph.Graph instance
    Transitively generate objects for the ``predicate`` relationship
    
    Generated objects belong to the depth first transitive closure of the
    ``predicate`` relationship starting at ``subject``.



In [160]:
list(g.triples(("https://fairscape.net/ark:59852/dataset-Simulated-unique-file-9LUEb5Y", None, None)))

[]

In [ ]:
# get all nodes
ds = datasetRDFLibNodes[0]
list(g.triples((ds, None, None)))

TypeError: unhashable type: 'list'

In [114]:
help(g.triples)

Help on method triples in module rdflib.graph:

triples(triple: '_TripleSelectorType') -> 'Generator[_TripleOrTriplePathType, None, None]' method of rdflib.graph.Graph instance
    Generator over the triple store
    
    Returns triples that match the given triple pattern. If triple pattern
    does not provide a context, all contexts will be searched.



In [98]:
print(len(datasetRDFLibNodes))
print(len(softwareRDFLibNodes))
print(len(computationRDFLibNodes))
print(len(rocrateRDFLibNode))

107
1
1
1


In [101]:
len(g.all_nodes())

936

In [107]:
len(g)

2625

In [108]:
# try deductive closure
from owlrl import DeductiveClosure, OWLRL_Semantics

In [109]:
DeductiveClosure(OWLRL_Semantics).expand(g)

In [110]:
len(g)

4995